## Principal Component Analysis (PCA) in Python

La Principal Component Analysis (PCA) è un'analisi multivariata utilizzata per ridurre la dimensionalità di un insieme di dati, identificando le componenti principali che rappresentano la maggior parte della variazione dei dati. In Python, la libreria scikit-learn fornisce una classe PCA che può essere utilizzata per eseguire la PCA.

In [31]:
# Non è necessario eseguire "pip install" delle librerie su Google Colab, in
# quanto sono già presenti nell'ambiente di sviluppo, dato il loro utilizzo
# intensivo in Machine Learning

import numpy as np
import sklearn as sn
import pandas as pd
import requests
import seaborn as sns

Per questo laboratorio, utilizzeremo il dataset *decathlon*, importandolo da locale.

In [ ]:
df = pd.read_csv ("decathlon.csv")

df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.select_dtypes(["object"]).describe()

Prima di applicare la PCA, è importante standardizzare le variabili numeriche.

In [ ]:
variabili_numeriche = list(df.columns[1:-1])
print(variabili_numeriche)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[variabili_numeriche])
scaled_data

Andiamo ad eseguire la PCA e a mostrare la varianza spiegata per ogni compontente principale mantenuta

In [ ]:
import matplotlib.pyplot as plt

pca = PCA().fit(scaled_data)

# Crea un grafico della varianza spiegata per ogni componente
plt.plot(range(1, pca.n_components_ + 1), pca.explained_variance_ratio_, marker='o')
plt.xlabel('Componenti della PCA')
plt.ylabel('Varianza spiegata')
plt.title("Risultati della PCA")
plt.show()

Ora applichiamo la PCA mantenendo le prime **2** componenti principali e

*   Voce elenco
*   Voce elenco

mappiamo i record nel nuovo spazio vettoriale generato dalla PCA.

In [64]:
# Applica la PCA per ridurre la dimensionalità dei dati
pca = PCA(n_components=2).fit(scaled_data)
pca_data = pca.transform(scaled_data)

In [ ]:
# Quantifichiamo  la percentuale di varianza spiegata da ciascuno delle componenti.
print(pca.explained_variance_ratio_)

In [ ]:
#Verifichiamo il coefficienti degli autovalori
eigenvalues = pca.explained_variance_
n_samples = scaled_data.shape[0]
cov_matrix = np.dot(scaled_data.T, scaled_data) / n_samples
for eigenvalue, eigenvector in zip(eigenvalues, pca.components_):
    print(np.dot(eigenvector.T, np.dot(cov_matrix, eigenvector)))
    print(eigenvalue)

Ora, creiamo un grafico delle osservazioni nella PCA

In [ ]:
# crea una mappa etichetta-->codice colore
unique_labels = np.unique(df["Competition"])
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))
label_to_color = dict(zip(unique_labels, colors))

# genera il grafico
fig, ax = plt.subplots()
for label in unique_labels:
    mask = (df["Competition"] == label).values
    ax.scatter(pca_data[mask, 0], pca_data[mask, 1], color=label_to_color[label], label=label)

ax.set_xlabel('Componente 1')
ax.set_ylabel('Componente 2')

ax.legend()

plt.show()

Possiamo visualizzare ogni attributo in un grafico 2D il cui asse x indica il  contributo rispetto prima componente e l'asse y rispetto alla seconda componente. Visualizziamo le proiezioni.

# Come identificare l'importanza di ogni caratteristica originale
Per identificare l'importanza di ogni feature di ciascun componente possiamo utilizzare utilizzare l'attributo ***components_***.

Il risultato è un array  in cui le righe rappresentano i componenti e le colonne rappresentano le caratteristiche originali.


In [73]:
print(abs(pca.components_))

[[0.32462717 0.34651451 0.28038448 0.26949298 0.31872084 0.3224588
  0.24267251 0.06639577 0.14621541 0.04809361 0.35809724 0.45001821]
 [0.11949296 0.25237393 0.46413427 0.27269463 0.42042142 0.15540646
  0.46898486 0.1528422  0.24403309 0.35562852 0.04480845 0.00118215]]


Questo valore ci dice "quanto" ogni caratteristica influenza ciascuna componente.

Quindi più alto è il valore (in valore assoluto), maggiore è l'influenza sulla componente principale.

Qui possiamo stimare che la dodicesima caratteristica ha spiegato il 45% della prima componente principale e la settima caratteristica ha spiegato il 47% della seconda componente principale.

Possiamo creare una rappresentazione più compatta delle informazioni di importanza degli attributi rispetto alle componenti

In [ ]:
ax = sns.heatmap(pca.components_,
                 cmap='YlGnBu',
                 yticklabels=[ "PCA"+str(x) for x in range(1,pca.n_components_+1)],
                 xticklabels=list(variabili_numeriche),
                 cbar_kws={"orientation": "vertical"})
ax.set_aspect("equal")

Valori positivi e negativi all'interno di pca.n_components_ riflettono la correlazione positiva e negativa delle variabili con le componenti principali.

In [ ]:
loadings = pca.components_
num_pc = pca.n_features_in_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
loadings_df
loadings_df['variable'] = variabili_numeriche
loadings_df = loadings_df.set_index('variable')
loadings_df
import matplotlib.pyplot as plt
ax = sns.heatmap(loadings_df, annot=True, cmap='Spectral')
plt.show()


In [ ]:
pcs = pca.components_
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])

for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :])):
    # plot line between origin and point (x, y)
    ax.plot([0, x], [0, y], color='k')
    # display the label of the point
    ax.text(x, y, df.columns[i], fontsize='10')

## Assignment

- Analizza il dataset IRIS:
  - Quante componenti sono necessarie per spiegare almeno il 95% della varianza nei dati?
  - In un nuovo spazio di rappresentazione, definito dalle componenti principali sopra determinate, i dati sono separabili rispetto alle classi?
  